<a href="https://colab.research.google.com/github/CyberXavier/MNIST/blob/main/%E6%89%8B%E5%86%99%E6%95%B0%E5%AD%97%E8%AF%86%E5%88%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 导入库
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets , transforms


In [ ]:
# 定义超参数
BATCH_SIZE = 64
EPOCHS = 10
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
# 构建pipline,对图像进行处理
pipline = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.1307,),(0.3081,))
])

In [ ]:
# 下载、加载数据集
from torch.utils.data import DataLoader

# 下载数据集
train_set = datasets.MNIST("data", train=True, download=True, transform=pipline)

test_set = datasets.MNIST("data", train=False, download=True, transform=pipline)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
# 加载数据
trian_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)

test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)


In [ ]:
# 构建模型
class Digit(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 10, 5) #1:单通道，10：输出通道，5：kernal:5*5
    self.conv2 = nn.Conv2d(10, 20, 3) 
    self.fc1 = nn.Linear(20*10*10, 500) # (输入通道, 输出通道)
    self.fc2 = nn.Linear(500, 10) #500:输入通道， 10：输出通道
  def forward(self, x):
    input_size = x.size(0) # (batch_size) * (1) * (28 * 28)
    x = self.conv1(x) # 输入:batch_size*1*28*28 输出：batch*10*24*24(28-5+1)
    x = F.relu(x) #保持shape不变
    x = F.max_pool2d(x,2,2) #入：batch*10*24*24 出：batch*10*12*12

    x = self.conv2(x) #入：batch*10*12*12 出：batch*20*10*10
    x = F.relu(x)

    x = x.view(input_size, -1) #拉平 -1：自动计算维度 20*10*10=2000

    x = self.fc1(x) #输入：batch*2000 输出：batch*500
    x = F.relu(x)

    x = self.fc2(x)

    output = F.log_softmax(x, dim=1) #计算分类后每个数值的概率
    
    return output


In [ ]:
# 定义优化器
model = Digit().to(DEVICE)

optimizer = optim.Adam(model.parameters()) #更新参数


In [ ]:
# 定义训练方法
def train_model(model, device, train_loader, optimizer, epoch):
  model.train()
  for batch_index,(data,target) in enumerate(train_loader):
    data, target = data.to(device),target.to(device)
    #梯度初始化为0
    optimizer.zero_grad()
    #得到结果
    output = model(data)
    #计算loss
    loss = F.cross_entropy(output,target)
    #反向传播
    loss.backward()
    #参数优化
    optimizer.step()
    if batch_index % 3000 == 0:
      print("Train Epoch : {}\t Loss : {:.6f}".format(epoch,loss.item()))

In [ ]:
# 定义测试方法
def test_model(model, device, test_loader):
  #模型验证
  model.eval()
  #正确率
  corrent = 0.0
  #测试损失
  test_loss = 0.0
  with torch.no_grad(): #不会计算梯度，也不会反向传播
    for data, target in test_loader:
      data, target = data.to(device), target.to(device)
      # 测试数据
      pre_output = model(data)
      #计算测试损失
      test_loss = F.cross_entropy(pre_output, target).item()
      # 找到概率值最大的下标
      pred = pre_output.max(1, keepdim = True)[1] # 0:值 ， 1:索引。这里找的是索引。
      # 累计正确图片个数
      corrent += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100.0 * corrent / len(test_loader.dataset)
    print("Test--Average loss：{:.4f}，Accuracy：{:.3f}\n".format(test_loss, accuracy))
    if accuracy > 99 :
      torch.save(model,'save.pt')

In [ ]:
# 调用方法7，8
for epoch in range(1,EPOCHS + 1):
  train_model(model, DEVICE, trian_loader, optimizer, epoch)
  test_model(model, DEVICE, test_loader)

Train Epoch : 1	 Loss : 0.000661
Test--Average loss：0.0000，Accuracy：98.910

Train Epoch : 2	 Loss : 0.030415
Test--Average loss：0.0000，Accuracy：99.100

Train Epoch : 3	 Loss : 0.053486
Test--Average loss：0.0000，Accuracy：99.020

Train Epoch : 4	 Loss : 0.000264
Test--Average loss：0.0000，Accuracy：99.040

Train Epoch : 5	 Loss : 0.000377
Test--Average loss：0.0000，Accuracy：99.200

Train Epoch : 6	 Loss : 0.000007
Test--Average loss：0.0000，Accuracy：98.930

Train Epoch : 7	 Loss : 0.012223
Test--Average loss：0.0000，Accuracy：98.840

Train Epoch : 8	 Loss : 0.000022
Test--Average loss：0.0000，Accuracy：98.970

Train Epoch : 9	 Loss : 0.000000
Test--Average loss：0.0000，Accuracy：99.000

Train Epoch : 10	 Loss : 0.000004
Test--Average loss：0.0000，Accuracy：98.910



In [ ]:
torch.load('save.pt')

Digit(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=2000, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=10, bias=True)
)